# 1. 데이터 전처리

In [1]:
import pandas as pd
import numpy as np
import re
from tqdm.auto import tqdm
import random

In [2]:
import torch
from torch.utils.data import Dataset
import torch.nn as nn

In [3]:
from scipy.stats import loguniform
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import itertools
from itertools import permutations
from transformers import (
    AutoTokenizer,
    AutoModel,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback
)

In [4]:
print(torch.cuda.is_available())

True


In [5]:
# from google.colab import drive
# drive.mount('/content/drive')

In [6]:
# %cd /content/drive/MyDrive/강의/AI를 위한 딥러닝/AI_DL_Project/code

In [7]:
# 데이터 로드
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')
submission = pd.read_csv('../data/sample_submission.csv')

In [8]:
train.info()
train.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7351 entries, 0 to 7350
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ID          7351 non-null   object
 1   sentence_0  7351 non-null   object
 2   sentence_1  7351 non-null   object
 3   sentence_2  7351 non-null   object
 4   sentence_3  7351 non-null   object
 5   answer_0    7351 non-null   int64 
 6   answer_1    7351 non-null   int64 
 7   answer_2    7351 non-null   int64 
 8   answer_3    7351 non-null   int64 
dtypes: int64(4), object(5)
memory usage: 517.0+ KB


,ID,sentence_0,sentence_1,sentence_2,sentence_3,answer_0,answer_1,answer_2,answer_3
0,TRAIN_0000,블록체인 기술은 투표 과정의 투명성을 크게 향상시킬 수 있다.,"이러한 특성은 유권자들에게 신뢰를 제공하며, 민주적 참여를 촉진하는 데 기여할 수 있다.",결과적으로 블록체인 기반의 투표 시스템은 공정하고 신뢰할 수 있는 선거 환경을 조성...,각 투표는 변경 불가능한 기록으로 저장되어 조작의 가능성을 원천적으로 차단한다.,0,3,1,2
1,TRAIN_0001,줄거리 자동 생성의 인공지능 알고리즘은 대량의 텍스트 데이터를 분석하여 핵심 정보를...,"결과적으로, 이러한 기술은 사용자에게 신속하고 효율적인 정보 전달을 가능하게 한다.",생성된 줄거리는 원본 텍스트의 의미를 유지하면서도 간결하게 요약된 형태로 제공된다.,"이 알고리즘은 자연어 처리 기술을 활용하여 문맥을 이해하고, 주요 사건과 등장인물을...",0,3,2,1
2,TRAIN_0002,"마지막으로, 키친타올을 보관할 때는 쉽게 접근할 수 있는 곳에 두어 낭비를 방지하는...",재사용 가능한 천이나 스펀지를 활용하면 키친타올의 필요성을 줄일 수 있다.,물기를 제거할 때는 가볍게 눌러주어 과도한 사용을 피할 수 있다.,키친타올을 절약하는 첫걸음은 필요한 양만큼만 사용하는 것이다.,3,2,1,0
3,TRAIN_0003,책의 페이지가 손상되지 않도록 수직으로 세워 두거나 평평하게 눕혀 보관하는 것이 좋다.,"정기적으로 먼지를 털어내고, 곰팡이나 해충의 발생 여부를 점검하는 것이 중요하다.",종이책은 직사광선이 닿지 않는 서늘하고 건조한 장소에 보관해야 한다.,"필요할 경우, 책을 보호하기 위해 커버를 씌우거나 전용 보관함에 넣는 방법도 고려할...",2,0,1,3
4,TRAIN_0004,"인공지능 모델은 반복적인 실험을 통해 지속적으로 학습하며, 이를 통해 발견의 정확성...",인공지능은 대량의 데이터를 분석하여 숨겨진 패턴과 상관관계를 발견하는 데 강력한 도...,"결국, 인공지능의 지원은 과학적 발견의 속도와 효율성을 혁신적으로 변화시킬 수 있는...",이러한 분석 결과는 연구자들에게 새로운 가설을 제시하고 실험 설계를 개선하는 데 기...,1,3,0,2


In [9]:
test.info()
test.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1780 entries, 0 to 1779
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ID          1780 non-null   object
 1   sentence_0  1780 non-null   object
 2   sentence_1  1780 non-null   object
 3   sentence_2  1780 non-null   object
 4   sentence_3  1780 non-null   object
dtypes: object(5)
memory usage: 69.7+ KB


,ID,sentence_0,sentence_1,sentence_2,sentence_3
0,TEST_0000,"자유 의지와 결정론은 서로 상충하는 개념으로 여겨지지만, 이 둘의 공존 가능성도 탐...","결정론은 모든 사건이 원인과 결과의 연쇄에 의해 발생한다고 주장하며, 이는 인간의 ...",그러나 인간의 인식과 선택 과정에서 나타나는 복잡성과 예측 불가능성은 자유 의지의 ...,"결국, 자유 의지와 결정론은 서로를 배제하기보다는, 인간 경험의 다양한 측면을 설명..."
1,TEST_0001,사회적 낙인은 개인의 자아 존중감에 부정적인 영향을 미친다.,"건강 불평등은 이러한 낙인으로 인해 더욱 심화되며, 특정 집단이 의료 서비스 접근에...","결국, 사회적 낙인과 건강 불평등은 서로 연결되어 있으며, 이를 해결하기 위한 포괄...","낙인으로 인해 사람들은 사회적 고립을 경험하고, 이는 정신적 및 신체적 건강에 악영..."
2,TEST_0002,글쓰기 능력을 키우기 위해서는 꾸준한 연습이 필수적이다.,"마지막으로, 독서를 통해 다른 작가들의 기법을 배우는 것은 창의력을 자극하는 데 도...",피드백을 받는 과정은 글의 질을 향상시키는 중요한 요소로 작용한다.,다양한 주제에 대해 글을 써보면 자신의 스타일과 강점을 발견할 수 있다.
3,TEST_0003,작은 공간에서도 효율적으로 사용할 수 있어 집안의 혼잡함을 줄여준다.,정기적으로 내용을 점검하면 필요 없는 물건을 정리할 수 있다.,각 칸을 활용하여 카테고리별로 물건을 나누면 찾기 쉬워진다.,다용도 수납함은 다양한 물건을 정리하는 데 유용하다.
4,TEST_0004,음악은 특정 문화의 가치와 전통을 반영하는 중요한 매체이다.,이러한 음악적 표현은 공동체의 소속감을 증진시키는 역할을 한다.,각 문화는 고유한 음악적 요소를 통해 정체성을 형성하고 강화한다.,"결국, 음악은 개인과 집단의 문화적 정체성을 이해하는 데 필수적인 요소로 작용한다."


In [10]:
submission.info()
submission.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1780 entries, 0 to 1779
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ID        1780 non-null   object
 1   answer_0  1780 non-null   int64 
 2   answer_1  1780 non-null   int64 
 3   answer_2  1780 non-null   int64 
 4   answer_3  1780 non-null   int64 
dtypes: int64(4), object(1)
memory usage: 69.7+ KB


,ID,answer_0,answer_1,answer_2,answer_3
0,TEST_0000,0,1,2,3
1,TEST_0001,0,1,2,3
2,TEST_0002,0,1,2,3
3,TEST_0003,0,1,2,3
4,TEST_0004,0,1,2,3


In [11]:
# 텍스트 정제
def clean_text(text):
  # 특수문자 제거
  text = re.sub(r'[^\w\s]', '', text)
  # 소문자 변환: 한글에는 무의미
  text = text.lower()
  # 불필요한 공백 제거
  text = ' '.join(text.split())
  return text

In [12]:
# 정제 전
print(train['sentence_0'][0])
print(test['sentence_0'][0])

블록체인 기술은 투표 과정의 투명성을 크게 향상시킬 수 있다.
자유 의지와 결정론은 서로 상충하는 개념으로 여겨지지만, 이 둘의 공존 가능성도 탐구할 가치가 있다.


In [13]:
# 텍스트 정제
for i in range(4):
    train[f'sentence_{i}'] = train[f'sentence_{i}'].apply(clean_text)
    test[f'sentence_{i}'] = test[f'sentence_{i}'].apply(clean_text)

In [14]:
# 정제 후
print(train['sentence_0'][0])
print(test['sentence_0'][0])

블록체인 기술은 투표 과정의 투명성을 크게 향상시킬 수 있다
자유 의지와 결정론은 서로 상충하는 개념으로 여겨지지만 이 둘의 공존 가능성도 탐구할 가치가 있다


In [15]:
# ✅ Pairwise 전처리 함수
def preprocess_pairwise(df):
    data = []
    for _, row in df.iterrows():
        sentences = [row[f'sentence_{i}'] for i in range(4)]
        answer = [row[f'answer_{i}'] for i in range(4)]
        ordered = [sentences[i] for i in answer]
        positive_pairs = [(ordered[i], ordered[i+1]) for i in range(3)]
        all_pairs = list(itertools.permutations(sentences, 2))
        for s1, s2 in all_pairs:
            label = 1 if (s1, s2) in positive_pairs else 0
            data.append({'sentence1': s1, 'sentence2': s2, 'label': label})
    return pd.DataFrame(data)

In [16]:
# 전처리 모듈 사용
pairwise_df = preprocess_pairwise(train)

In [17]:
pairwise_df.info()
pairwise_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88212 entries, 0 to 88211
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   sentence1  88212 non-null  object
 1   sentence2  88212 non-null  object
 2   label      88212 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 2.0+ MB


,sentence1,sentence2,label
0,블록체인 기술은 투표 과정의 투명성을 크게 향상시킬 수 있다,이러한 특성은 유권자들에게 신뢰를 제공하며 민주적 참여를 촉진하는 데 기여할 수 있다,0
1,블록체인 기술은 투표 과정의 투명성을 크게 향상시킬 수 있다,결과적으로 블록체인 기반의 투표 시스템은 공정하고 신뢰할 수 있는 선거 환경을 조성...,0
2,블록체인 기술은 투표 과정의 투명성을 크게 향상시킬 수 있다,각 투표는 변경 불가능한 기록으로 저장되어 조작의 가능성을 원천적으로 차단한다,1
3,이러한 특성은 유권자들에게 신뢰를 제공하며 민주적 참여를 촉진하는 데 기여할 수 있다,블록체인 기술은 투표 과정의 투명성을 크게 향상시킬 수 있다,0
4,이러한 특성은 유권자들에게 신뢰를 제공하며 민주적 참여를 촉진하는 데 기여할 수 있다,결과적으로 블록체인 기반의 투표 시스템은 공정하고 신뢰할 수 있는 선거 환경을 조성...,1


In [18]:
# 클래스 균형 확인
pairwise_df['label'].value_counts()

label
0    66159
1    22053
Name: count, dtype: int64

# 2. 데이터셋 구성

In [19]:
MAX_TOKEN_LENGTH = 128

In [20]:
# 데이터셋 클래스 정의
class SentencePairDataset(Dataset):
    """
    문장 쌍을 받아 BERT 입력 형식으로 변환하는 PyTorch Dataset 클래스

    Args:
        texts (List[Tuple[str, str]]): (문장1, 문장2) 형태의 튜플 리스트
        labels (List[int]): 문장 순서가 맞는지 여부를 나타내는 정수 레이블 (예: 0 또는 1)
        tokenizer (transformers.PreTrainedTokenizer): HuggingFace 토크나이저 객체
        max_length (int): 토큰 최대 길이 (default=128)
    """
    def __init__(self, dataframe, tokenizer, max_length=MAX_TOKEN_LENGTH):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        encoding = self.tokenizer(
            row['sentence1'],
            row['sentence2'],
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )

        item = {
            'input_ids': encoding['input_ids'][0],
            'attention_mask': encoding['attention_mask'][0],
            'labels': torch.tensor(row['label'], dtype=torch.long)
        }

        if 'token_type_ids' in encoding:
            item['token_type_ids'] = encoding['token_type_ids'][0]

        return item

# 3. 모델 아키텍쳐

In [21]:
class EnhancedSentencePairModel(nn.Module):
    def __init__(self, model_name, num_labels=2, class_weights=None, 
                 hidden_size=2048, num_attention_heads=16, dropout_rate=0.1):
        super(EnhancedSentencePairModel, self).__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        
        # 1. 더 큰 hidden size 사용
        self.classifier = nn.Sequential(
            nn.Linear(self.bert.config.hidden_size * 2, hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_size, hidden_size // 2),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_size // 2, num_labels)
        )
        
        # 2. Attention 메커니즘 추가
        self.attention = nn.MultiheadAttention(
            embed_dim=self.bert.config.hidden_size,
            num_heads=num_attention_heads,
            dropout=dropout_rate
        )
        
        # 3. Layer Normalization 추가
        self.layer_norm = nn.LayerNorm(self.bert.config.hidden_size)
        
        if class_weights is not None:
            self.loss_fn = nn.CrossEntropyLoss(weight=class_weights)
        else:
            self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, input_ids, attention_mask=None, token_type_ids=None, labels=None):
        outputs = self.bert(input_ids=input_ids,
                          attention_mask=attention_mask,
                          token_type_ids=token_type_ids,
                          output_hidden_states=True,
                          return_dict=True)

        # 4. 더 많은 레이어 활용
        hidden_states = outputs.hidden_states
        last_five = torch.stack(hidden_states[-5:])
        avg_hidden = torch.mean(last_five, dim=0)
        
        # 5. Self-attention 적용
        avg_hidden = avg_hidden.permute(1, 0, 2)
        attn_output, _ = self.attention(avg_hidden, avg_hidden, avg_hidden)
        attn_output = attn_output.permute(1, 0, 2)
        
        # 6. Layer Normalization 적용
        attn_output = self.layer_norm(attn_output)
        
        # 7. [CLS] 토큰과 마지막 토큰의 representation 결합
        cls_output = attn_output[:, 0]
        last_token_output = attn_output[:, -1]
        combined_output = torch.cat([cls_output, last_token_output], dim=-1)
        
        # 8. 분류기 통과
        logits = self.classifier(combined_output)

        loss = None
        if labels is not None:
            loss = self.loss_fn(logits, labels)

        return {'loss': loss, 'logits': logits}

# 4. 학습 코드

In [22]:
# 토크나이저 초기화
tokenizer = AutoTokenizer.from_pretrained('monologg/koelectra-base-v3-discriminator')

In [23]:
# 데이터 분할: 학습/검증 (예: 8:2)
train_df, val_df = train_test_split(pairwise_df, test_size=0.2, stratify=pairwise_df['label'], random_state=42)

# dataset 생성
train_dataset = SentencePairDataset(train_df, tokenizer, max_length=MAX_TOKEN_LENGTH)
val_dataset = SentencePairDataset(val_df, tokenizer, max_length=MAX_TOKEN_LENGTH)

In [24]:
# GPU 사용 여부 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [25]:
# 모델 인스턴스 생성
model = EnhancedSentencePairModel("monologg/koelectra-base-v3-discriminator").to(device)

In [26]:
def compute_metrics(pred):
    logits, labels = pred
    preds = logits.argmax(axis=1)

    return {
        'accuracy': accuracy_score(labels, preds),
        'f1': f1_score(labels, preds),
        'precision': precision_score(labels, preds),
        'recall': recall_score(labels, preds)
    }

In [28]:
# 학습 인자 설정
training_args = TrainingArguments(
    output_dir='./koelectra_results',     # 체크포인트와 결과 저장 폴더
    num_train_epochs=15,                   # 학습 epoch 수
    per_device_train_batch_size=64,       # 학습 시 디바이스당 배치 사이즈
    per_device_eval_batch_size=128,        # 검증 시 디바이스당 배치 사이즈
    warmup_steps=500,                     # 워밍업 단계에서 학습률 선형 증가
    weight_decay=0.02,                    # AdamW optimizer의 weight decay 값
    logging_dir='./koelectra_logs',       # 로깅 파일 저장 경로를 절대 경로로 변경
    logging_steps=10,                     # 로그를 몇 스텝마다 남길지
    save_strategy='epoch',                # epoch마다 체크포인트 저장
    eval_strategy='epoch',
    seed=42,
    learning_rate=1.6e-4,  # 학습률 명시적 설정
    gradient_accumulation_steps=2,  # 그래디언트 누적 추가
    fp16=True,  # mixed precision training 활성화
    lr_scheduler_type='cosine_with_restarts',  # cosine learning rate scheduling
    max_grad_norm=1.0,  # gradient clipping

    # EarlyStoppingCallback 필수 옵션
    load_best_model_at_end=True,           # 학습이 끝난 뒤 최적 모델 불러오기
    metric_for_best_model='accuracy',      # 대회 평가 산식 monitor
    greater_is_better=True                 # 메트릭이 클수록 좋은 경우 True
)

# 트레이너 초기화
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
   )

# 학습 실행
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.362400,0.345632,0.833135,0.622951,0.715926,0.551349
2,0.253500,0.280031,0.870430,0.736818,0.748538,0.725459
3,0.237700,0.290959,0.877175,0.745866,0.772595,0.720925
4,0.175300,0.274815,0.893499,0.771606,0.831761,0.719565
5,0.159600,0.284495,0.885394,0.760256,0.796918,0.726819
6,0.119400,0.266008,0.912770,0.822675,0.836457,0.809340
7,0.072400,0.359798,0.916114,0.828664,0.846700,0.811381
8,0.045200,0.320385,0.919572,0.838805,0.840619,0.836998
9,0.037600,0.321713,0.921669,0.841804,0.850173,0.833598
10,0.023600,0.349707,0.922122,0.837396,0.875959,0.802086


TrainOutput(global_step=8265, training_loss=0.11611378208344776, metrics={'train_runtime': 6858.0905, 'train_samples_per_second': 154.348, 'train_steps_per_second': 1.205, 'total_flos': 0.0, 'train_loss': 0.11611378208344776, 'epoch': 14.973708068902992})

In [29]:
# epoch 마지막 또는 전체 학습 후 평가
eval_results = trainer.evaluate()
print("Validation Results:", eval_results)

Validation Results: {'eval_loss': 0.6003636717796326, 'eval_accuracy': 0.9294337697670464, 'eval_f1': 0.8576329331046312, 'eval_precision': 0.8652514997692663, 'eval_recall': 0.8501473588755384, 'eval_runtime': 35.7186, 'eval_samples_per_second': 493.944, 'eval_steps_per_second': 3.864, 'epoch': 14.973708068902992}


In [30]:
# 모델 저장
trainer.save_model("./koelectra_results/final_model")

# 5. 하이퍼파라미터 튜닝

In [57]:
param_grid = {
    # 모델 구조 관련
    'hidden_size': [1024, 1536, 2048],
    'num_attention_heads': [12, 16],
    'dropout_rate': [0.1, 0.2],
    
    # 학습 관련
    'learning_rate': [3e-5, 4e-5, 5e-5],
    'weight_decay': [0.01, 0.02],
    'warmup_steps': [1000],
    'batch_size': [16, 32]
}

In [54]:
def sample_dataset(dataset, sample_size=5000, random_state=42):
    """데이터셋에서 랜덤 샘플링"""
    # DataFrame에서 랜덤 샘플링
    sampled_data = dataset.data.sample(n=sample_size, random_state=random_state)
    # 새로운 데이터셋 생성
    return SentencePairDataset(sampled_data, dataset.tokenizer)

In [55]:
from itertools import product

def grid_search(param_grid, train_dataset, val_dataset, sample_size=1000):
    # 데이터셋 샘플링
    sampled_train = sample_dataset(train_dataset, sample_size)
    sampled_val = sample_dataset(val_dataset, sample_size // 5)  # 검증셋은 더 작게
    
    print(f"Sampled dataset sizes - Train: {len(sampled_train)}, Val: {len(sampled_val)}")
    
    # 모든 파라미터 조합 생성
    param_combinations = [dict(zip(param_grid.keys(), v)) 
                         for v in product(*param_grid.values())]
    
    results = []
    
    for params in tqdm(param_combinations, desc="Grid Search"):
        # 모델 초기화
        model = EnhancedSentencePairModel(
            model_name="monologg/koelectra-base-v3-discriminator",
            hidden_size=params['hidden_size'],
            num_attention_heads=params['num_attention_heads'],
            dropout_rate=params['dropout_rate']
        )
        
        # 학습 인자 설정 (체크포인트 저장 비활성화)
        training_args = TrainingArguments(
            output_dir='./temp',
            num_train_epochs=3,  # 샘플링된 데이터셋이므로 epochs도 줄임
            per_device_train_batch_size=params['batch_size'],
            per_device_eval_batch_size=params['batch_size'] * 2,
            warmup_steps=params['warmup_steps'],
            weight_decay=params['weight_decay'],
            learning_rate=params['learning_rate'],
            fp16=True,
            save_strategy='no',
            load_best_model_at_end=False,
            metric_for_best_model='accuracy',
            greater_is_better=True
        )
        
        # 트레이너 초기화
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=sampled_train,
            eval_dataset=sampled_val,
            compute_metrics=compute_metrics
        )
        
        # 학습 및 평가
        trainer.train()
        eval_results = trainer.evaluate()
        
        # 결과 기록
        result = {
            'params': params,
            'accuracy': eval_results['eval_accuracy'],
            'loss': eval_results['eval_loss']
        }
        results.append(result)
        
        print(f"\nParameters: {params}")
        print(f"Accuracy: {eval_results['eval_accuracy']:.4f}")
        print(f"Loss: {eval_results['eval_loss']:.4f}")
        print("-" * 50)
    
    # 결과를 데이터프레임으로 변환
    df_results = pd.DataFrame(results)
    
    # 파라미터 컬럼 분리
    param_df = pd.DataFrame([r['params'] for r in results])
    df_results = pd.concat([param_df, df_results[['accuracy', 'loss']]], axis=1)
    
    # 결과 정렬
    df_results = df_results.sort_values('accuracy', ascending=False)
    
    return df_results

In [ ]:
# 그리드 서치 실행
results_df = grid_search(param_grid, train_dataset, val_dataset, sample_size=5000)

# 결과 출력
print("\nTop 10 configurations:")
print(results_df.head(10))

# 최적의 파라미터 선택
best_params = results_df.iloc[0].to_dict()  # 가장 높은 accuracy를 가진 파라미터 조합

# 결과 저장
results_df.to_csv('grid_search_results.csv', index=False)

In [60]:
results_df.info()
results_df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 144 entries, 130 to 25
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   hidden_size          144 non-null    int64  
 1   num_attention_heads  144 non-null    int64  
 2   dropout_rate         144 non-null    float64
 3   learning_rate        144 non-null    float64
 4   weight_decay         144 non-null    float64
 5   warmup_steps         144 non-null    int64  
 6   batch_size           144 non-null    int64  
 7   accuracy             144 non-null    float64
 8   loss                 144 non-null    float64
dtypes: float64(5), int64(4)
memory usage: 11.2 KB


,hidden_size,num_attention_heads,dropout_rate,learning_rate,weight_decay,warmup_steps,batch_size,accuracy,loss
130,2048,16,0.1,0.00005,0.02,1000,16,0.797,0.457752
142,2048,16,0.2,0.00005,0.02,1000,16,0.797,0.436326
102,2048,12,0.1,0.00004,0.02,1000,16,0.797,0.446884
34,1024,16,0.1,0.00005,0.02,1000,16,0.797,0.441270
138,2048,16,0.2,0.00004,0.02,1000,16,0.796,0.431237


In [61]:
best_params

{'hidden_size': 2048.0,
 'num_attention_heads': 16.0,
 'dropout_rate': 0.1,
 'learning_rate': 5e-05,
 'weight_decay': 0.02,
 'warmup_steps': 1000.0,
 'batch_size': 16.0,
 'accuracy': 0.797,
 'loss': 0.45775243639945984}

In [ ]:
# 최적 파라미터로 최종 모델 학습
final_model = EnhancedSentencePairModel(
    model_name="monologg/koelectra-base-v3-discriminator",
    hidden_size=2048,
    num_attention_heads=16,
    dropout_rate=0.1
)

# 최종 학습 인자 설정
final_training_args = TrainingArguments(
    output_dir='./koelectra_results/final_model',
    num_train_epochs=15,
    per_device_train_batch_size=2048,
    per_device_eval_batch_size=2048 * 2,
    warmup_steps=1000,
    weight_decay=0.02,
    learning_rate=5e-05,
    fp16=True,
    eval_strategy='epoch',
    save_strategy='epoch',
    save_total_limit=3,  # 최근 3개 체크포인트만 저장
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    greater_is_better=True,
    lr_scheduler_type='cosine_with_restarts',
    max_grad_norm=1.0,
    logging_dir='./koelectra_logs',
    logging_steps=100,
)

# 최종 모델 학습
final_trainer = Trainer(
    model=final_model,
    args=final_training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3, early_stopping_threshold=0.001)]
)

final_trainer.train()

In [ ]:
# 최종 평가
final_results = final_trainer.evaluate()
print("\nFinal evaluation results:")
print(f"Accuracy: {final_results['eval_accuracy']:.4f}")
print(f"Loss: {final_results['eval_loss']:.4f}")

# 6. 평가 및 추론

In [31]:
def predict_order(sent1, sent2, model, tokenizer, device='cpu'):
    model.eval()
    inputs = tokenizer(
        sent1,
        sent2,
        return_tensors='pt',
        padding=True,
        truncation=True,
        max_length=MAX_TOKEN_LENGTH
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.softmax(outputs['logits'], dim=1)

    return predictions, predictions.argmax().item()

# 7. 페어와이즈 예측 함수

In [32]:
def pair_score(s1, s2):
    probs, _ = predict_order(s1, s2, model, tokenizer, device)
    return probs[0][1].item()  # label=1 확률

# 8. 순열별 점수 계산 후 최적 순서 선택

In [33]:
pred_orders = []
pred_scores = []

# 테스트 샘플 수 만큼 반복
for idx, (_, row) in enumerate(tqdm(test.iterrows(),
                                     total=len(test),
                                     desc="문장 순서 예측")):
    sents = [row[f'sentence_{i}'] for i in range(4)]

    best_perm, best_score = None, -1.0
    # 24가지 순열 전부 탐색
    for perm in permutations(range(4), 4):
        # 인접 페어 3개의 점수 합산
        score = sum(pair_score(sents[perm[i]], sents[perm[i+1]])
                    for i in range(3))
        if score > best_score:
            best_score, best_perm = score, perm

    pred_orders.append(best_perm)
    pred_scores.append(best_score)

    # # 매 50개마다 로그 출력
    # if (idx + 1) % 50 == 0:
    #     print(f"[{idx+1}/{len(test)}] 현재까지 최고 평균 스코어: {np.mean(pred_scores):.4f}")

문장 순서 예측:   0%|          | 0/1780 [00:00<?, ?it/s]

# 9. submission 파일 작성

In [34]:
for idx, perm in enumerate(pred_orders):
    for i, p in enumerate(perm):
        submission.loc[idx, f'answer_{i}'] = p

submission.to_csv('koelectra_results/submission.csv', index=False)
print("✅ submission.csv 파일이 생성되었습니다.")

✅ submission.csv 파일이 생성되었습니다.


In [35]:
submission.head()

,ID,answer_0,answer_1,answer_2,answer_3
0,TEST_0000,1,2,0,3
1,TEST_0001,0,3,1,2
2,TEST_0002,0,3,2,1
3,TEST_0003,3,2,0,1
4,TEST_0004,0,2,1,3
